In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
asc_model_path = "../models/asc_model/checkpoint-885"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
asc_model = AutoModelForSequenceClassification.from_pretrained(asc_model_path)

In [ ]:
sentence = "I charge it at night and skip taking the cord with me because of the good battery life."
aspects = ["card"]

for aspect in aspects:
    inputs = tokenizer(
        sentence,
        aspect,   
        return_tensors="pt",
        truncation=True,
        padding=True
    )
    outputs = asc_model(**inputs)
    predicted_class = outputs.logits.argmax(dim=-1).item()
    
    label_map = {3 :"positive", 1: "negative", 2: "neutral"}  
    print(f"Aspect: {aspect} → {label_map[predicted_class]}")

Aspect: card → neutral


In [ ]:
from transformers import pipeline, AutoTokenizer

asc_model_path = "../models/asc_model/checkpoint-885"
ate_model_path = "../models/ate_model"

asc_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 

ate_pipeline = pipeline(
    "token-classification",
    model=ate_model_path,
    aggregation_strategy="simple"
)

asc_pipeline = pipeline(
    "text-classification",
    model=asc_model_path,
    tokenizer=asc_tokenizer
)

label_map = {
    1: "negative",
    2: "neutral",
    3: "positive"
}

def analyze_sentence(sentence):
    aspects = ate_pipeline(sentence)
    results = {}

    for asp in aspects:
        aspect_term = asp['word']

        input_text = f"Aspect: {aspect_term}. Sentence: {sentence}"
        prediction = asc_pipeline(input_text)[0]

        predicted_class = int(prediction['label'].replace("LABEL_", ""))
        polarity = label_map.get(predicted_class, "unknown")

        results[aspect_term] = polarity

    return results


sentence = "this shit hard disk is so slow."
print(analyze_sentence(sentence))


Device set to use cpu
Device set to use cpu


{'hard disk': 'negative'}


In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification

# Load ATE model
MODEL_ATE_PATH = "../models/ate_model"
ate_tokenizer = AutoTokenizer.from_pretrained(MODEL_ATE_PATH)
ate_model = AutoModelForTokenClassification.from_pretrained(MODEL_ATE_PATH)
id2label = ate_model.config.id2label

# -------------------------------
# Load ASC model 
ASC_MODEL_PATH = "../models/asc_model/checkpoint-885"
asc_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
asc_model = AutoModelForSequenceClassification.from_pretrained(ASC_MODEL_PATH)

# Step 1: Extract Aspects (ATE)
def extract_aspects(text):
    tokens = ate_tokenizer(text.split(), is_split_into_words=True, return_tensors="pt")
    with torch.no_grad():
        outputs = ate_model(**tokens)
    predictions = torch.argmax(outputs.logits, dim=2)

    word_ids = tokens.word_ids()
    aspects, current = [], []

    for idx, word_id in enumerate(word_ids):
        if word_id is None:
            continue
        token = tokens.tokens()[idx]
        label = id2label[predictions[0][idx].item()]

        if label.startswith("B-Aspect"):
            if current:
                aspects.append(" ".join(current))
                current = []
            current.append(token)
        elif label.startswith("I-Aspect"):
            current.append(token)
        else:
            if current:
                aspects.append(" ".join(current))
                current = []

    if current:
        aspects.append(" ".join(current))

    aspects = [asp.replace("##", "") for asp in aspects]
    return aspects

# Step 2: Classify Sentiment (ASC)
def classify_sentiment(sentence, aspect):
    inputs = asc_tokenizer(
        sentence,
        aspect,
        return_tensors="pt",
        truncation=True,
        padding=True
    )
    with torch.no_grad():
        outputs = asc_model(**inputs)
    predicted_class = outputs.logits.argmax(dim=-1).item()

    
    label_map = {
        1: "negative",
        2: "neutral",
        3: "positive"
    }
    return label_map[predicted_class]


# Demo
if __name__ == "__main__":
    sentence = "The design looks nice and the performance is fast."

    print("\nSentence:", sentence)

    # Run ATE
    aspects = extract_aspects(sentence)
    print("Extracted Aspects:", aspects)

    # Run ASC
    for aspect in aspects:
        sentiment = classify_sentiment(sentence, aspect)
        print(f"Aspect: {aspect} → {sentiment}")



Sentence: The design looks nice and the performance is fast.
Extracted Aspects: ['performance']
Aspect: performance → positive
